### Assignment-01

#### 1. 第一节课的作业如下：

1. 复现课程的代码， BFS 路线搜索与文本自动生成；
2. 回答以下问题，并将问题发送至deeplearning.nlp.zh@gmail.com中：
```
    2.1. what do you want to acquire in this course？
    2.2. what problems do you want to solve？
    2.3. what’s the advantages you have to finish you goal?
    2.4. what’s the disadvantages you need to onvercome to finish you goal?
    2.5. How will you plan to study in this course period?
```
3. 完成以下联系，进一步体会 Data Driving的编程思想，完成你的第一个较为复杂的对话机器人。

#### 2. 如何提交

1. 代码 + 此 jupyter 相关，提交至自己的 github 中；
2. 第2问，整理发送至deeplearning.nlp.zh@gmail.com邮箱。

#### 3. 截止时间

此次作业截止时间为 2019.4.6日



## 基于模式匹配的对话机器人实现

### Pattern Match

机器能否实现对话，这个长久以来是衡量机器人是否具有智能的一个重要标志。 Alan Turing早在其文中就提出过一个测试机器智能程度的方法，该方法主要是考察人类是否能够通过对话内容区分对方是机器人还是真正的人类，如果人类无法区分，我们就称之为具有”智能“。而这个测试，后来被大家叫做”图灵测试“，之后也被翻拍成了一步著名电影，叫做《模拟游戏》。 



既然图灵当年以此作为机器是否具备智能的标志，这项任务肯定是复杂的。自从 1960s 开始，诸多科学家就希望从各个方面来解决这个问题，直到如今，都只能解决一部分问题。 目前对话机器人的建立方法有很多，今天的作业中，我们为大家提供一共快速的基于模板的对话机器人配置方式。

此次作业首先希望大家能够读懂这段程序的代码，其次，在此基于我们提供的代码，**能够把它改造成汉语版本，实现对话效果。**

```
Pattern: (我想要A)
Response: (如果你有 A，对你意味着什么呢？)

Input: (我想要度假)
Response: (如果你有度假，对你意味着什么呢？)
```

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成  "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

In [1924]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

is_variable() 判断是否是以?开头，且其余全为字符

In [1925]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

pat_match 首字符相等向后比较，直至找到？ 返回True  不等返回False

目的是找到每个变量对应什么单词

### 例如

In [1926]:
pat_match('I want ?X'.split(), "I want holiday".split())

True

In [1927]:
pat_match('I have dreamed a ?X'.split(), "I dreamed about dog".split())

False

In [1928]:
pat_match('I dreamed about ?X'.split(), "I dreamed about dog".split())

True

### 获得匹配的变量

以上的函数能够判断两个 pattern 是不是相符，但是我们更加希望的是获得每个variable对应的是什么值。

我们对程序做如下修改:

In [1929]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0] #直至找到？ ，返回？X和 对应位置单词；  递归基
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])#递归

In [1930]:
pattern = 'I want ?X'.split()
saying = "I want holiday".split()

In [1931]:
pat_match(pattern, saying)

('?X', 'holiday')

In [1932]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

('?X', '2+2')

但是，如果我们的 Pattern 中具备两个变量，那么以上程序就不能解决了，我们可以对程序做如下修改: 


In [1933]:
def pat_match(pattern, saying):
    if not pattern or not saying: return [] #pattern, saying有一个为空，返回空list
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:]) #加号后面 递归 即找到？X后继续向后查找，解决多个？X问题
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

于是，我们可以获得： 

In [1934]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

如果我们知道了每个变量对应的是什么，那么我们就可以很方便的使用我们定义好的模板进行替换：

为了方便接下来的替换工作，我们新建立两个函数，一个是把我们解析出来的结果变成一个 dictionary，一个是依据这个 dictionary 依照我们的定义的方式进行替换。

In [1935]:
def pat_to_dict(patterns): #将pat_match返回结果从list变成dictionary
    return {k: v for k, v in patterns}

In [1936]:
def subsitite(rule, parsed_rules):
    if not rule: return []  #rule为空时 返回[] 递归基
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules) #rule中一个一个替换 rule即pat_to_dict返回值

get(a,b)返回指定key为a 的value    若没找到a则return b 

In [1937]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())

In [1938]:
got_patterns

[('?X', 'iPhone')]

In [1939]:
subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

为了将以上输出变成一句话，也很简单，我们使用 Python 的 join 方法即可： 

In [1940]:
john_pat = pat_match('?P needs ?X'.split(), "John needs resting".split())

In [1941]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [1942]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())

In [1943]:
subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))

['Why', 'does', 'John', 'need', 'vacation', '?']

In [1944]:
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

那么如果我们现在定义一些patterns，就可以实现基于模板的对话生成了:

In [1945]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [1946]:
import random

def pat_match_all(saying,rules,key):
    if not key : return 'not match'
    #print(key[0])
    pattern = pat_match(key[0].split(),saying.split())
    #print( pattern )
    if pattern: return pattern , random.choice(rules[key[0]])
    return pat_match_all(saying,rules,key[1:])

In [1947]:
def get_response(saying, rules):
    key = list(rules.keys())
    pattern , response = pat_match_all(saying,rules,key)
    return ' '.join(subsitite(response.split(), pat_to_dict(pattern)))
    
    
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    #pass
    

In [1948]:
get_response('My mother told me something',defined_patterns)

'Talk about more about your mother'

### Segment Match

我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？ 

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable

In [1949]:
def is_pattern_segment(pattern):#找到?* 且都是字符
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [1950]:
is_pattern_segment('?*P')

True

In [1951]:
from collections import defaultdict   #有什么用？

然后我们把之前的 ```pat_match```程序改写成如下， 主要是增加了 ``` is_pattern_segment ```的部分. 

In [1952]:
fail = [True,None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):#？X规则
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):#？*X规则
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:#重复的跳过
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

这段程序里比较重要的一个新函数是 ```segment_match```，这个函数输入是一个以 ```segment_pattern```开头的模式，尽最大可能进行，匹配到这个*边长*的变量对于的部分。

In [1953]:
def segment_match(pattern, saying):#要求pattern以?*开头
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')#为了显示成?X形式

    if not rest: return (seg_pat, saying), len(saying)    #如果?X后面没有内容，则saying全为?X
    
    for i, token in enumerate(saying):#匹配rest[0]对应saying位置
        if rest[0] == token :#and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
        
    #return (seg_pat, None), 0
    return fail



In [1954]:
segment_match('?*P is very good'.split(), "My dog and" .split())#有bug

[True, None]

现在，我们就可以做到以下的匹配模式了: 

In [1955]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

如果我们继续定义一些模板，我们进行匹配，就能够进行更加复杂的问题了: 

In [1956]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [1957]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

#pat_match_with_seg('?*Y need ?*X'.split(), 
#                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [1958]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

 我们会发现，pat_to_dict在这个场景下会有有一点小问题，没关系，修正一些: 

In [1959]:
def pat_to_dict(patterns):#字典压平
    print(patterns)
    s=[]
    for i in patterns:
        if i!=True and i!=None:
            s.append(i)
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in s}#k是索引

In [1960]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

[('?X', ['an', 'iPhone'])]


['Why', 'do', 'you', 'neeed', 'an iPhone']

如果我们定义这样的一个模板:

In [1961]:
("?*X hello ?*Y", "Hi, how do you do")

('?*X hello ?*Y', 'Hi, how do you do')

In [1962]:
subsitite("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

[('?X', ['I', 'am', 'mike,'])]


['Hi,', 'how', 'do', 'you', 'do?']

### 现在是你的时间了

In [1963]:
#我们给大家一些例子: 
    
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

### 问题1

编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [1964]:
def get_response(saying,  response_rules):
    if not saying or not response_rules: return []
    key = list(response_rules.keys())
    random.shuffle(key)
    #print(key)
    pattern , response = pat_match_all(saying,response_rules,key)
    #print(pattern)
    return ' '.join(subsitite(response.split(), pat_to_dict(pattern)))

def pat_match_all(saying,rules,key):
    if not key : return 'not match'
    #print(key[0],saying)
    pattern = pat_match_with_seg(key[0].split(),saying.split())
    #print( pattern )
    if pattern: return pattern , random.choice(rules[key[0]])
    return pat_match_all(saying,rules,key[1:])

In [1965]:
get_response('I was chen', rules)

[True, True, None]


'Hi, how do you do?'

### 问题2

改写以上程序，将程序变成能够支持中文输入的模式。
*提示*: 你可以需用用到 jieba 分词

In [1966]:
import re

In [1967]:
def seg_char(sent):
    """
    把句子按字分开，不破坏英文结构
    """
    pattern = re.compile(r'([\u4e00-\u9fa5])')
    chars = pattern.split(sent)
    chars = [w for w in chars if len(w.strip())>0]
    return chars

def seg_char(sent):
    """
    把句子按字分开，不破坏英文结构
    """
    pattern = re.compile(r'([\u4e00-\u9fa5])')
    chars = pattern.split(sent)
    chars = [w for w in chars if len(w.strip())>0]
    return chars
--------------------- 
作者：Autuming 
来源：CSDN 
原文：https://blog.csdn.net/renyuanfang/article/details/86487367 
版权声明：本文为博主原创文章，转载请附上博文链接！

In [1968]:
def cout_ENCH (sent):
    s = []
    j = (sent).split()
    for i in j :
        s = s + seg_char(i)
    return s
#seg_char(('你觉得?y有什么意义呢？').split())

In [1969]:
cout_ENCH ('你家里除了?y还有谁?')

['你', '家', '里', '除', '了', '?y', '还', '有', '谁', '?']

将上文split换成cout_ENCH ()

In [1970]:
def get_response_ENCH(saying, response_rules):
    key = list(response_rules.keys())
    random.shuffle(key)
    #print(key)
    pattern , response = pat_match_all_ENCH(saying,response_rules,key)
    #print(pattern , response)
    return ' '.join(subsitite(cout_ENCH(response), pat_to_dict(pattern)))

def pat_match_all_ENCH(saying,rules,key):
    #if not key : return 'not match'
    #print(key[0],saying)
    pattern = pat_match_with_seg(cout_ENCH(key[0]),cout_ENCH(saying))
    #print( pattern )
    if pattern: return pattern , random.choice(rules[key[0]])
    return pat_match_all(saying,rules,key[1:])

In [1971]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}


In [1979]:
get_response_ENCH('他们是小狗吗？ ', rule_responses)

[True, True, None]


'你 不 想 要 ?y 吗 ？'

segment_match函数匹配有问题，暂未找到解决办法。

### 问题3

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

In [1973]:
rule_responses = {
    '?*x 为什么 *y': ['我也不清楚', '请百度一下'],
    '?*x 是什么颜色': ['红色', '黄色', '无色']
}

### 问题4

1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？ 

优点：模式固定，针对特定问题能够在可选范围内找到对应答案
缺点：只能解决单一类型问题，对于未定义模式无法回答，不能够举一反三。
改进方法：1、增加更多的response_rules 2、构建更通用的response_rules 3、放弃通过公式映射的方式，更换解决问题的思路

2. 什么是数据驱动？数据驱动在这个程序里如何体现？

数据驱动：仅更换数据，不更改代码
在这个程序里如何体现：对应不同的response_rules，可以在不更改算法只更换response_rules的情况下解决问题

3. 数据驱动与 AI 的关系是什么？ 

AI是数据驱动算法，但不是所有的数据驱动算法都是AI



一些参考 pattern

In [1974]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}
